<a href="https://colab.research.google.com/github/Sanskriti-Raghav/Solution-for-deepfake-images-and-videos/blob/main/Deepfake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y build-essential cmake
!pip install dlib torchvision torchaudio torch tqdm matplotlib seaborn opencv-python-headless
!wget -q http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 -f shape_predictor_68_face_landmarks.dat.bz2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import torch

In [ ]:
!unzip -q "/content/archive (1).zip" -d /content/

In [ ]:
!ls /content/Dataset

Test  Train  Validation


In [ ]:
!ls /content/Dataset/Train

Fake  Real


In [ ]:
!ls -R /content/Dataset

Streaming output truncated to the last 5000 lines.
fake_12600.jpg	fake_16137.jpg	fake_1997.jpg	fake_5532.jpg  fake_9069.jpg
fake_12601.jpg	fake_16138.jpg	fake_1998.jpg	fake_5533.jpg  fake_906.jpg
fake_12602.jpg	fake_16139.jpg	fake_1999.jpg	fake_5534.jpg  fake_9070.jpg
fake_12603.jpg	fake_1613.jpg	fake_199.jpg	fake_5535.jpg  fake_9071.jpg
fake_12604.jpg	fake_16140.jpg	fake_19.jpg	fake_5536.jpg  fake_9072.jpg
fake_12605.jpg	fake_16141.jpg	fake_1.jpg	fake_5537.jpg  fake_9073.jpg
fake_12606.jpg	fake_16142.jpg	fake_2000.jpg	fake_5538.jpg  fake_9074.jpg
fake_12607.jpg	fake_16143.jpg	fake_2001.jpg	fake_5539.jpg  fake_9075.jpg
fake_12608.jpg	fake_16144.jpg	fake_2002.jpg	fake_553.jpg   fake_9076.jpg
fake_12609.jpg	fake_16145.jpg	fake_2003.jpg	fake_5540.jpg  fake_9077.jpg
fake_1260.jpg	fake_16146.jpg	fake_2004.jpg	fake_5541.jpg  fake_9078.jpg
fake_12610.jpg	fake_16147.jpg	fake_2005.jpg	fake_5542.jpg  fake_9079.jpg
fake_12611.jpg	fake_16148.jpg	fake_2006.jpg	fake_5543.jpg  fake_907.jpg
fake_12612

In [ ]:
!ls /content/Dataset/Train
!ls /content/Dataset/Validation
!ls /content/Dataset/Test

Fake  Real
Fake  Real
Fake  Real


In [ ]:
import os, dlib, cv2, numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# -------------------
# Config
# -------------------
IMG_SIZE = 64        # downscale from 256x256 to 64x64
PATCH_SIZE = 8
EMBED_DIM = 64
NUM_HEADS = 4
NUM_LAYERS = 2
SYM_DIM = 50
NUM_CLASSES = 2
BATCH_SIZE = 16
EPOCHS = 5
LR = 3e-4

DATA_ROOT = "/content/Dataset"
LANDMARKS = "shape_predictor_68_face_landmarks.dat"
CACHE_DIR = Path("/content/sym_cache"); CACHE_DIR.mkdir(exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------
# Facial Symmetry Extractor
# -------------------
class FacialSymmetryExtractor:
    def __init__(self, landmarks=LANDMARKS, dim=SYM_DIM):
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(landmarks)
        self.dim = dim

    def _calc(self, pts):
        pairs = [(0,16),(1,15),(2,14),(3,13),(4,12),(5,11),(6,10),(7,9),
                 (17,26),(18,25),(19,24),(20,23),(21,22),
                 (36,45),(37,44),(38,43),(39,42),(40,47),(41,46),
                 (31,35),(32,34),
                 (48,54),(49,53),(50,52),(58,56),(59,55)]
        feats = []
        cx = np.mean(pts[:,0])
        for l,r in pairs:
            ld, rd = abs(pts[l,0]-cx), abs(pts[r,0]-cx)
            feats.extend([ld/(rd+1e-6), abs(pts[l,1]-pts[r,1])])
        le, re = pts[36:42], pts[42:48]
        lw, rw = np.linalg.norm(le[3]-le[0]), np.linalg.norm(re[3]-re[0])
        lh, rh = np.linalg.norm(le[1]-le[5]), np.linalg.norm(re[1]-re[5])
        feats.extend([lw/(rw+1e-6), lh/(rh+1e-6)])
        if len(feats)<self.dim: feats.extend([0]*(self.dim-len(feats)))
        return np.array(feats[:self.dim],dtype=np.float32)

    def extract(self, arr):
        gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
        faces = self.detector(gray)
        if not faces: return np.zeros(self.dim,dtype=np.float32)
        largest = max(faces,key=lambda f:f.width()*f.height())
        lm = self.predictor(gray,largest)
        pts = np.array([(lm.part(i).x,lm.part(i).y) for i in range(68)],dtype=np.float32)
        return self._calc(pts)

sym_extractor = FacialSymmetryExtractor()

# -------------------
# Dataset
# -------------------
class FaceDataset(Dataset):
    def __init__(self, root, split, transform):
        self.items=[]
        self.transform=transform
        for label,cls in [(0,"Real"),(1,"Fake")]: # Corrected capitalization
            search_path = Path(root)/split/cls
            print(f"Searching in: {search_path}")
            for p in search_path.glob("*.jpg"):
                self.items.append((str(p),label))
        print(f"{split} samples found: {len(self.items)}")
        if len(self.items) == 0:
            print(f"Error: No items found for split {split}. Please check the dataset path and file extensions.")
        print(f"Length of self.items at the end of __init__ for {split}: {len(self.items)}")


    def __len__(self): return len(self.items)

    def __getitem__(self,idx):
        path,label=self.items[idx]
        try:
            img=Image.open(path).convert("RGB")
            arr=np.array(img)
            key=Path(path).stem
            cache_file=CACHE_DIR/(key+".npy")
            if cache_file.exists():
                sym=np.load(cache_file)
            else:
                sym=sym_extractor.extract(arr)
                np.save(cache_file,sym)
            return self.transform(img), torch.tensor(sym,dtype=torch.float32), torch.tensor(label)
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            return None, None, None # Return None for problematic samples


# -------------------
# Model
# -------------------
class TransformerBlock(nn.Module):
    def __init__(self,d,h):
        super().__init__()
        self.ln1=nn.LayerNorm(d); self.attn=nn.MultiheadAttention(d,h,batch_first=True)
        self.ln2=nn.LayerNorm(d); self.mlp=nn.Sequential(nn.Linear(d,d*2),nn.GELU(),nn.Linear(d*2,d))
    def forward(self,x):
        h=x; x=self.ln1(x); x,_=self.attn(x,x,x); x=h+x; h=x; x=self.ln2(x); return h+self.mlp(x)

class ToyViT(nn.Module):
    def __init__(self,img,patch,d,h,layers,sym_dim,num_classes):
        super().__init__()
        self.num_patches=(img//patch)**2; self.patch_dim=3*patch*patch
        self.proj=nn.Linear(self.patch_dim,d)
        self.cls=nn.Parameter(torch.zeros(1,1,d))
        self.pos=nn.Parameter(torch.randn(1,self.num_patches+1,d)*0.02)
        self.blocks=nn.ModuleList([TransformerBlock(d,h) for _ in range(layers)])
        self.norm=nn.LayerNorm(d)
        self.sym_proj=nn.Linear(sym_dim,d)
        self.fc=nn.Linear(d*2,num_classes)
        self.patch=patch

    def forward(self,x,sym):
        B=x.size(0)
        p=self.patch
        patches=x.unfold(2,p,p).unfold(3,p,p).contiguous().view(B,3,-1,p,p).permute(0,2,1,3,4).reshape(B,-1,3*p*p)
        x=self.proj(patches)
        cls=self.cls.expand(B,-1,-1)
        x=torch.cat([cls,x],1)+self.pos
        for blk in self.blocks: x=blk(x)
        cls=self.norm(x)[:,0]
        sym=self.sym_proj(sym)
        return self.fc(torch.cat([cls,sym],1))

# -------------------
# Train / Eval
# -------------------
def run():
    transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.ToTensor()])
    train_ds=FaceDataset(DATA_ROOT,"Train",transform)
    val_ds=FaceDataset(DATA_ROOT,"Validation",transform)
    test_ds=FaceDataset(DATA_ROOT,"Test",transform)

    print(f"Train dataset size: {len(train_ds)}")
    print(f"Validation dataset size: {len(val_ds)}")
    print(f"Test dataset size: {len(test_ds)}")

    # Check if samples can be accessed
    try:
        for i in range(min(5, len(train_ds))):
            img, sym, label = train_ds[i]
            print(f"Successfully loaded sample {i} from train_ds")
    except Exception as e:
        print(f"Error accessing sample from train_ds: {e}")


    train_loader=DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
    val_loader=DataLoader(val_ds,batch_size=BATCH_SIZE)
    test_loader=DataLoader(test_ds,batch_size=BATCH_SIZE)

    model=ToyViT(IMG_SIZE,PATCH_SIZE,EMBED_DIM,NUM_HEADS,NUM_LAYERS,SYM_DIM,NUM_CLASSES).to(device)
    opt=torch.optim.Adam(model.parameters(),lr=LR)
    crit=nn.CrossEntropyLoss()

    for ep in range(EPOCHS):
        model.train(); tloss,tacc=0,0
        for x,sym,y in tqdm(train_loader,desc=f"Epoch {ep+1}"):
            # Filter out problematic samples
            if x is None:
                continue

            x,sym,y=x.to(device),sym.to(device),y.to(device)
            opt.zero_grad(); out=model(x,sym); loss=crit(out,y); loss.backward(); opt.step()
            tloss+=loss.item(); tacc+=(out.argmax(1)==y).sum().item()
        print(f"Epoch {ep+1}: loss {tloss/len(train_loader):.3f} acc {tacc/len(train_ds):.3f}")
        model.eval(); correct=0
        with torch.no_grad():
            for x,sym,y in val_loader:
                 # Filter out problematic samples
                if x is None:
                    continue
                out=model(x.to(device),sym.to(device)); correct+=(out.argmax(1)==y.to(device)).sum().item()
        print(f"  Val acc {correct/len(val_ds):.3f}")

    # Test
    model.eval(); correct=0
    with torch.no_grad():
        for x,sym,y in test_loader:
             # Filter out problematic samples
            if x is None:
                continue
            out=model(x.to(device),sym.to(device)); correct+=(out.argmax(1)==y.to(device)).sum().item()
    print(f"Test accuracy: {correct/len(test_ds):.3f}")

if __name__=="__main__":
  run()

Searching in: /content/Dataset/Train/Real
Searching in: /content/Dataset/Train/Fake
Train samples found: 140002
Length of self.items at the end of __init__ for Train: 140002
Searching in: /content/Dataset/Validation/Real
Searching in: /content/Dataset/Validation/Fake
Validation samples found: 39428
Length of self.items at the end of __init__ for Validation: 39428
Searching in: /content/Dataset/Test/Real
Searching in: /content/Dataset/Test/Fake
Test samples found: 10905
Length of self.items at the end of __init__ for Test: 10905
Train dataset size: 140002
Validation dataset size: 39428
Test dataset size: 10905
Successfully loaded sample 0 from train_ds
Successfully loaded sample 1 from train_ds
Successfully loaded sample 2 from train_ds
Successfully loaded sample 3 from train_ds
Successfully loaded sample 4 from train_ds


Epoch 1: 100%|██████████| 8751/8751 [30:55<00:00,  4.72it/s]


Epoch 1: loss 289.713 acc 0.733
  Val acc 0.732


Epoch 2: 100%|██████████| 8751/8751 [05:03<00:00, 28.85it/s]


Epoch 2: loss 271.610 acc 0.789
  Val acc 0.769


...

Epoch 3: loss 220.244 acc 0.803
  Val acc 0.788


Epoch 4: 100%|██████████| 8751/8751 [04:36<00:00, 31.68it/s]


Epoch 4: loss 171.840 acc 0.814
  Val acc 0.783


Epoch 5: 100%|██████████| 8751/8751 [04:38<00:00, 31.41it/s]


Epoch 5: loss 134.908 acc 0.824
  Val acc 0.794
Test accuracy: 0.773
